In [1]:
# Import libraries
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import RMSprop
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

import torch.optim as optim

import random

# Function to set the seed
#def set_seed(seed):
#    torch.manual_seed(seed)
#    torch.cuda.manual_seed(seed)
#    torch.cuda.manual_seed_all(seed)
#    np.random.seed(seed)
#    random.seed(seed)
#    torch.backends.cudnn.deterministic = True
#    torch.backends.cudnn.benchmark = False

#set_seed(42)

torch.manual_seed(0)
np.random.seed(0)

# Create training and test sets
features, target = make_classification(n_classes=2, n_features=10,
n_samples=1000)
features_train, features_test, target_train, target_test = train_test_split(
features, target, test_size=0.1, random_state=1)




# Convert data to PyTorch tensors
x_train = torch.from_numpy(features_train).float()
y_train = torch.from_numpy(target_train).float().view(-1, 1)
x_test = torch.from_numpy(features_test).float()
y_test = torch.from_numpy(target_test).float().view(-1, 1)


# Define a neural network using `Sequential`
class SimpleNeuralNet(nn.Module):
    def __init__(self):
        super(SimpleNeuralNet, self).__init__()
        self.sequential = torch.nn.Sequential(
            torch.nn.Linear(10, 16),
            torch.nn.ReLU(),
            torch.nn.Linear(16,16),
            torch.nn.ReLU(),
            torch.nn.Linear(16, 1),
            torch.nn.Sigmoid()
        )
    def forward(self, x):
        x = self.sequential(x)
        return x

# Initialize neural network
network = SimpleNeuralNet()
# Define loss function, optimizer
criterion = nn.BCELoss()
optimizer = RMSprop(network.parameters())

# Define data loader
train_data = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_data, batch_size=100, shuffle=True)
# Compile the model using torch 2.0's optimizer
network = torch.compile(network)

#import torch._dynamo
#torch._dynamo.config.suppress_errors = True

# Train neural network
epochs = 3
for epoch in range(epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = network(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
    print("Epoch:", epoch+1, "\tLoss:", loss.item())
# Evaluate neural network

with torch.no_grad():
    output = network(x_test)
    test_loss = criterion(output, y_test)
    test_accuracy = (output.round() == y_test).float().mean()
    print("Test Loss:", test_loss.item(), "\tTest Accuracy:",
        test_accuracy.item())

W0618 09:38:52.250000 8808 torch\_dynamo\convert_frame.py:824] WON'T CONVERT forward C:\Users\tdiak_000\AppData\Local\Temp\ipykernel_22240\2126035803.py line 57 
W0618 09:38:52.250000 8808 torch\_dynamo\convert_frame.py:824] due to: 
W0618 09:38:52.250000 8808 torch\_dynamo\convert_frame.py:824] Traceback (most recent call last):
W0618 09:38:52.250000 8808 torch\_dynamo\convert_frame.py:824]   File "C:\Users\tdiak_000\anaconda3\Lib\site-packages\torch\_dynamo\convert_frame.py", line 786, in _convert_frame
W0618 09:38:52.250000 8808 torch\_dynamo\convert_frame.py:824]     result = inner_convert(
W0618 09:38:52.250000 8808 torch\_dynamo\convert_frame.py:824]              ^^^^^^^^^^^^^^
W0618 09:38:52.250000 8808 torch\_dynamo\convert_frame.py:824]   File "C:\Users\tdiak_000\anaconda3\Lib\site-packages\torch\_dynamo\convert_frame.py", line 400, in _convert_frame_assert
W0618 09:38:52.250000 8808 torch\_dynamo\convert_frame.py:824]     return _compile(
W0618 09:38:52.250000 8808 torch\_dyn

Epoch: 1 	Loss: 0.19006989896297455
Epoch: 2 	Loss: 0.14092369377613068
Epoch: 3 	Loss: 0.03935525193810463
Test Loss: 0.06877756118774414 	Test Accuracy: 0.9700000286102295
